In [1]:
import torch

C:\Users\Mehedi\PycharmProjects\TorchEnvironment\venvTorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# install opencv "pip install opencv-python" 
import cv2 

# distance from camera to object(face) measured 
# centimeter 
Known_distance = 175 
  
# width of face in the real world or Object Plane 
# centimeter 
Known_width = 14.15 #14.15 for men 13.51 for women

# Colors 
GREEN = (0, 255, 0) 
RED = (0, 0, 255) 
WHITE = (255, 255, 255) 
BLACK = (0, 0, 0) 

image_path = "font profile.jpg"

# defining the fonts 
fonts = cv2.FONT_HERSHEY_COMPLEX 

# face detector object 
face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml") 

# focal length finder function 
def Focal_Length_Finder(measured_distance, real_width, width_in_rf_image): 
    focal_length = (width_in_rf_image * measured_distance) / real_width 
    return focal_length #2000 is threshold

# distance estimation function 
def Distance_finder(Focal_Length, real_face_width, face_width_in_frame): 
    distance = (real_face_width * Focal_Length)/face_width_in_frame 
    # return the distance 
    return distance 

def face_data(image): 

    face_width = 0 # making face width to zero 

    # converting color image to gray scale image 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

    # detecting face in the image 
    faces = face_detector.detectMultiScale(gray_image, 1.3, 5) 

    # looping through the faces detect in the image 
    # getting coordinates x, y , width and height 
    for (x, y, h, w) in faces: 

        # draw the rectangle on the face 
        cv2.rectangle(image, (x, y), (x+w, y+h), GREEN, 2) 
        # getting face width in the pixels 
        face_width = w 

    # return the face width in pixel 
    print('face width function',face_width) 
    return face_width 


# find the face width(pixels) in the real word human image
ref_image_face_width = 534.80 #534.80 for male 510.61 for female

# get the focal by calling "Focal_Length_Finder" 
# face width in reference(pixels), 
# Known_distance(centimeters), 
# known_width(centimeters) 
Focal_length_found = Focal_Length_Finder(Known_distance, Known_width, ref_image_face_width) 

print("focal lenght", Focal_length_found) 

# # show the reference image 
# cv2.imshow("ref_image", ref_image) 

person_image = cv2.imread(image_path)
# calling face_data function to find 
# the width of face(pixels) in the frame 
face_width_in_frame = face_data(person_image) 
print("face width", face_width_in_frame)
# check if the face is zero then not 
# find the distance 
if face_width_in_frame != 0: 
    # finding the distance by calling function 
    # Distance finder function need 
    # these arguments the Focal_Length, 
    # Known_width(centimeters), 
    # and Known_distance(centimeters) 
    Distance = Distance_finder(Focal_length_found, Known_width, face_width_in_frame) 
    print(f"Distance: {round(Distance,2)} CM")
    # draw line as background of text 
    cv2.line(person_image, (30, 30), (230, 30), RED, 32) 
    cv2.line(person_image, (30, 30), (230, 30), BLACK, 28) 
    # Drawing Text on the screen 
    cv2.putText(person_image, f"Distance: {round(Distance,2)} CM", (30, 35), fonts, 0.6, GREEN, 2) 
# show the frame on the screen 
cv2.imwrite('distance measured.jpg', person_image)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()


focal lenght 6614.134275618373
face width function 342
face width 342
Distance: 273.65 CM


In [3]:
from ultralytics import YOLO

In [4]:
## load a pretrained YOLOv8n model
model = YOLO('yolov8m-pose.pt')

In [5]:
results = model(source=image_path, show=False, conf=0.70, save=True)


image 1/1 C:\Users\Mehedi\Jupyter Notebook Project\TMTSoftProject\Cloth size estimation\face distance measurement\font profile.jpg: 640x384 1 person, 103.9ms
Speed: 22.0ms preprocess, 103.9ms inference, 119.8ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs\pose\predict2


In [6]:
bounding_box = results[0].boxes.xyxy[0]  # Detections for the first image in batch 
x1, y1, x2, y2 = bounding_box.tolist() 
width = x2 - x1
height = y2 - y1
width, height

(954.0, 3480.0)

In [7]:
bounding_box

tensor([ 996., 1016., 1950., 4496.], device='cuda:0')

In [8]:
height, Distance, Focal_length_found

(3480.0, 273.6549707602339, 6614.134275618373)

In [9]:
# Convert dimensions from pixels to centimeters
actual_width_cm = (width * Distance) / (Focal_length_found-1720)
actual_height_cm = (height * Distance) / (Focal_length_found-1720)
actual_width_cm, actual_height_cm

(53.342803323939734, 194.58381086720155)

In [10]:
## side measurements
side_image = "side profle.jpg"
side_results = model(source=side_image, show=False, conf=0.70, save=True)
side_bounding_box = side_results[0].boxes.xyxy[0]  # Detections for the first image in batch 
x1, y1, x2, y2 = side_bounding_box.tolist() 
side_width = x2 - x1 
actual_side_width_cm = ((side_width * Distance) / (Focal_length_found-1720))/2.75
actual_side_width_cm


image 1/1 C:\Users\Mehedi\Jupyter Notebook Project\TMTSoftProject\Cloth size estimation\face distance measurement\side profle.jpg: 640x384 1 person, 238.6ms
Speed: 9.7ms preprocess, 238.6ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs\pose\predict2


13.439905849866273

In [11]:
tensor_values = results[0].keypoints.xy[0]
# Extracting specific keypoints based on the request
specific_keypoints_tensors = {
    "Left Shoulder": tensor_values[5],
    "Right Shoulder": tensor_values[6],
    "Left Elbow": tensor_values[7],
    "Right Elbow": tensor_values[8],
    "Left Wrist": tensor_values[9],
    "Right Wrist": tensor_values[10],
    "Left Hip": tensor_values[11],
    "Right Hip": tensor_values[12]
}
hip_adjustment = torch.norm(specific_keypoints_tensors['Left Hip'] - specific_keypoints_tensors['Right Hip']).item()
hip_adjustment = hip_adjustment+(hip_adjustment/2)
shoulder_adjustment = torch.norm(specific_keypoints_tensors['Left Shoulder'] - specific_keypoints_tensors['Right Shoulder']).item()
shoulder_adjustment = shoulder_adjustment+(shoulder_adjustment/3)
## wrist calculation.
shoulder_to_neck =  shoulder_adjustment/2
shouldertoelbow = torch.norm(specific_keypoints_tensors['Left Shoulder'] - specific_keypoints_tensors['Left Elbow']).item()
elbowtowrist = torch.norm(specific_keypoints_tensors['Left Elbow'] - specific_keypoints_tensors['Left Wrist']).item()
necktowirst = shoulder_to_neck+shouldertoelbow+elbowtowrist
# Calculate distances using Pythagorean theorem
pixel_distances = {
    'chest':shoulder_adjustment ,
    'waist': hip_adjustment,
    'sleeve': necktowirst,
}

In [12]:
# Convert distances from pixels to cm
measurements_cm = {key: (value * Distance) / (Focal_length_found-1720) for key, value in pixel_distances.items()}
measurements_cm['neck'] = 2*(measurements_cm['waist']-10.16)
measurements_cm['chest'] = 2*(measurements_cm['chest']+actual_side_width_cm)
measurements_cm['waist'] = 2*(measurements_cm['waist']+actual_side_width_cm)
measurements_cm['height'] = 100
measurements_cm

{'chest': 109.58997158474216,
 'waist': 90.02491230391912,
 'sleeve': 76.28472562779463,
 'neck': 42.82510060418658,
 'height': 100}

In [13]:
def calculate_size(height, chest, sleeve, neck):
    if height < 170:
        if chest < 90:
            return 'XS'
        elif 90 <= chest < 96:
            return 'S'
        elif 96 <= chest < 102:
            return 'M'
        elif 102 <= chest < 108:
            return 'L'
        else:
            return 'XL'
    elif 170 <= height < 180:
        if 90 <= chest < 96:
            return 'S'
        elif 96 <= chest < 102:
            return 'M'
        elif 102 <= chest < 108:
            return 'L'
        elif 108 <= chest < 114:
            return 'XL'
        else:
            return 'XXL'
    else:
        if 96 <= chest < 102:
            return 'M'
        elif 102 <= chest < 108:
            return 'L'
        elif 108 <= chest < 114:
            return 'XL'
        elif 114 <= chest < 120:
            return 'XXL'
        else:
            return 'XXXL'


In [14]:
# Calculate T-shirt size
tshirt_size = calculate_size(measurements_cm['height'], measurements_cm['chest'], measurements_cm['sleeve'], measurements_cm['neck'])

print("T-shirt size:", tshirt_size)

T-shirt size: XL


In [15]:

# Conversion factor from cm to inches
cm_to_inch = 1 / 2.54

# Converting all measurements to inches
measurements_inch = {key: value * cm_to_inch for key, value in measurements_cm.items()}
measurements_inch


{'chest': 43.14565810422919,
 'waist': 35.442878859810676,
 'sleeve': 30.033356546375835,
 'neck': 16.860275828419912,
 'height': 39.370078740157474}